In [ ]:
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('tools/')

from pyfm import pylibfm 

from sklearn.model_selection import KFold


In [ ]:
trainFile = 'round1_ijcai_18_train_20180301.txt'
testFile = 'round1_ijcai_18_test_a_20180301.txt'
dataDir = '../../Data/advertisment/Raw/'

In [ ]:
dfTrain = pd.read_table(dataDir+trainFile,sep=' ')
dfTest = pd.read_table(dataDir+testFile,sep=' ')

In [ ]:
print(dfTrain.dtypes)
print(dfTrain.shape)

In [ ]:
featInput = []
keyList = ['instance_id','item_id','user_id','context_id','shop_id','is_trade']
for var in dfTrain.columns:
    if not var in keyList and dfTrain[var].dtypes != 'object':
        featInput.append(var)
print(featInput)
print(len(featInput))

<font color=#0099ff size=5 face="黑体">处理日期</font>

In [ ]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return value.tm_hour


In [ ]:
dfTrain['context_timestamp_hour'] = dfTrain['context_timestamp'].map(timestamp_datetime)
try: featInput.remove('context_timestamp')
except: print('context_timestamp has been deleted')
if not 'context_timestamp_hour' in featInput:
    featInput.append('context_timestamp_hour')

<font color=#0099ff size=5 face="黑体">One Hot变形</font>

In [ ]:
dummyInput = []
removeInput = []
for var in featInput:
    if 'int' in str(dfTrain[var].dtypes):
        print(var)
        dfTrain[var].replace({-1:np.nan},inplace=True)
        dfTmp = pd.get_dummies(dfTrain[var],var)
        dfTrain = pd.concat([dfTrain,dfTmp],axis=1)
        try:
            removeInput.append(var)
            dummyInput +=dfTmp.columns.tolist()
        except:
            print('Removed')
for var in removeInput:
    featInput.remove(var)
featInput+=dummyInput

In [ ]:
print(dfTrain.columns)
print(dfTrain.shape)

<font color=#0099ff size=5 face="黑体">CV 创建模型输入</font>

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
for trainIndex,testIndex in cv.split(dfTrain[keyList]):
    trainX,testX = dfTrain.loc[trainIndex,featInput],dfTrain.loc[testIndex,featInput]
    trainY,testY = dfTrain.loc[trainIndex,'is_trade'],dfTrain.loc[testIndex,'is_trade']

In [ ]:
trainX = dfTrain[dummyInput[0:21]].to_dict('records')

In [ ]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
trainX =  v.fit_transform(trainX)

In [ ]:
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="classification", initial_learning_rate=0.001, learning_rate_schedule="optimal")


In [ ]:
fm.fit(trainX, trainY)

In [ ]:
from sklearn.feature_extraction import DictVectorizer
train = [
{"user": "1", "item": "5", "age": 19},
{"user": "2", "item": "43", "age": 33},
{"user": "3", "item": "20", "age": 55},
{"user": "4", "item": "10", "age": 20},
]
v = DictVectorizer()
X = v.fit_transform(train)
print(X.toarray())

In [ ]:
type(X)

In [ ]:
fm = pywFM.FM(task='classification', num_iter=5)

In [ ]:
trainX.values

In [ ]:

model = tffm.TFFMClassifier(
    order=6,
    rank=10,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    n_epochs=100,
    batch_size=20000,
    init_std=0.001,
    input_type='dense'
)
model.fit(trainX.values, trainY.values, show_progress=True)